In [12]:
%load_ext autoreload
%autoreload 2

In [45]:
import urban_dictionary
import logging
import pickle
from scipy import stats
import pandas as pd
from tqdm.notebook import tqdm
from collections import OrderedDict
from functools import partial
from concurrent.futures import ThreadPoolExecutor
from multiprocessing.pool import ThreadPool
import itertools
import numpy as np

In [2]:
logging.basicConfig(level=logging.INFO)
session = urban_dictionary.get_session(throttle=0.1, expiry = (7*24*3600))

In [ ]:
"""
all_urls = urban_dictionary_scraper.fetch_all_word_urls(session)
with open("all_urls.pickle", "wb") as f:
    pickle.dump(all_urls, f, pickle.HIGHEST_PROTOCOL)
"""

In [3]:

with open("all_urls.pickle", "rb") as f:
    to_fetch = pickle.load(f)
    
with open("all_words.pickle", "rb") as f:
    already_done = pickle.load(f)
    for key in already_done.keys():
        del to_fetch[key]
        
done = 100 * len(already_done) / (len(already_done) + len(to_fetch))
print(f"Done {done:.2f} percent")

Done 71.25 percent


In [ ]:
t = ThreadPool(5)
#with ThreadPoolExecutor(max_workers=5) as executor:
try:
    fetch_all_definitions(session, to_fetch, already_done, save_interval=10000, executor=t)    
finally:
    t.terminate()
    t.join()

In [64]:
defns = pd.DataFrame(
    [
        [e.word, e.meaning, e.examples[0], e.creation_epoch, e.upvotes, e.downvotes]
        for e in itertools.chain.from_iterable(e.definitions for e in already_done.values())
    ],
    columns=["word", "meaning", "example", "creation_epoch", "upvotes", "downvotes"]
)

TypeError: 'float' object cannot be interpreted as an integer

In [65]:
(defns["meaning"].str.len() + defns["example"].str.len()).quantile(np.linspace(0.01, 1, 100))

0.01       38.00
0.02       47.00
0.03       54.00
0.04       59.00
0.05       63.00
          ...   
0.96      743.00
0.97      833.00
0.98      977.00
0.99     1266.51
1.00    91521.00
Length: 100, dtype: float64

In [61]:
lng_defs = defns[defns["meaning"].str.len() > 985]
(lng_defs["upvotes"] + lng_defs["downvotes"]).describe()

count     29628.000000
mean        257.874713
std        1731.727996
min           0.000000
25%          10.000000
50%          44.000000
75%         162.000000
max      230221.000000
dtype: float64

In [62]:
lng_defs = defns[defns["meaning"].str.len() < 985]
(lng_defs["upvotes"] + lng_defs["downvotes"]).describe()

count    2.940324e+06
mean     7.464941e+01
std      8.058769e+02
min      0.000000e+00
25%      2.000000e+00
50%      8.000000e+00
75%      3.100000e+01
max      6.271090e+05
dtype: float64